Problem Statement:
A company wants to do chatbot that working 24/7 can serve customer on their Whatsapp chat.

In the dataset, we have words, classes and documents

Let’s begin:

In [33]:
# Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
#load the intents.json file from your local device
from google.colab import files
files.upload()

Saving intentss.json to intentss.json


{'intentss.json': b'{\r\n        "intents": [\r\n                {\r\n                        "tag": "selamat",\r\n                        "patterns": [\r\n                          "Hi",\r\n                          "How are you",\r\n                          "Is anyone there?",\r\n                          "Hello",\r\n                          "Good day",\r\n                          "Assalamualaikum",\r\n                          "Hey",\r\n                          "Greetings",\r\n                          "What\'s up",\r\n                          "Hola"\r\n                        ],\r\n                        "responses": [\r\n                          "Hello, thanks for visiting",\r\n                          "Waalaikumsalam, thanks for visiting",\r\n                          "Waalaikumsalam, Good to see you again",\r\n                          "Waalaikumsalam, Hi there, how can I help?",\r\n                          "Hey! Nice to see you",\r\n                          "Greetings

In [38]:
# import our chat-bot intents file
with open('intentss.json') as json_data:
    intents = json.load(json_data)

In [39]:
intents

{'intents': [{'tag': 'selamat',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Assalamualaikum',
    'Hey',
    'Greetings',
    "What's up",
    'Hola'],
   'responses': ['Hello, thanks for visiting',
    'Waalaikumsalam, thanks for visiting',
    'Waalaikumsalam, Good to see you again',
    'Waalaikumsalam, Hi there, how can I help?',
    'Hey! Nice to see you',
    'Greetings! How can I assist you today?',
    'Hola! What brings you here?',
    "Hi! How's your day going?",
    'Hello! How can I be of service?',
    'Hey there! What can I do for you?'],
   'context_set': ''},
  {'tag': 'jumpa lagi',
   'patterns': ['Bye',
    'See you later',
    'Jumpa lagi kawan',
    'Goodbye',
    'Take care',
    'Farewell',
    'Until next time',
    'Adios',
    'So long',
    'Have a great day'],
   'responses': ['See you later, thanks for visiting',
    'Ok Jumpa lagi, kerana datang',
    'Have a nice day',
    'Bye! Come back again soon.',

In [40]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [41]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

63 documents
9 classes ['about', 'chatbot', 'connect', 'jumpa lagi', 'location', 'movies', 'neural networks', 'selamat', 'terima kasih']
120 unique stemmed words ["'re", "'s", '.', 'a', 'about', 'account', 'activ', 'address', 'adio', 'ani', 'anyon', 'appreci', 'are', 'assalamualaikum', 'autoencod', 'base', 'behind', 'built', 'bunch', 'bye', 'can', 'care', 'chatbot', 'cheer', 'cnn', 'concept', 'connect', 'countri', 'creat', 'creator', 'day', 'deep', 'develop', 'doe', 'explain', 'farewel', 'favourit', 'feedforward', 'function', 'geograph', 'give', 'good', 'goodby', 'grate', 'great', 'greet', 'have', 'hello', 'help', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'is', 'it', 'jumpa', 'kawan', 'lagi', 'later', 'learn', 'lifesav', 'link', 'locat', 'long', 'lstm', 'made', 'me', 'media', 'million', 'more', 'movi', 'name', 'network', 'neural', 'next', 'of', 'one', 'out', 'owe', 'part', 'perceptron', 'person', 'place', 'reach', 'recommend', 'rnn', 'role', 'see', 'situat', 'so', 'social', 'some', 'speci

In [42]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-42-0222230fb184>:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [43]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=9, verbose=1)
model.save("model.pkl")

Epoch 1/1000
7/7 [==============================] - 0s 4ms/step - loss: 3.9937e-05 - accuracy: 1.0000
Epoch 2/1000
7/7 [==============================] - 0s 4ms/step - loss: 3.9754e-05 - accuracy: 1.0000
Epoch 3/1000
7/7 [==============================] - 0s 4ms/step - loss: 3.9591e-05 - accuracy: 1.0000
Epoch 4/1000
7/7 [==============================] - 0s 4ms/step - loss: 3.9438e-05 - accuracy: 1.0000
Epoch 5/1000
7/7 [==============================] - 0s 4ms/step - loss: 3.9245e-05 - accuracy: 1.0000
Epoch 6/1000
7/7 [==============================] - 0s 3ms/step - loss: 3.9095e-05 - accuracy: 1.0000
Epoch 7/1000
7/7 [==============================] - 0s 4ms/step - loss: 3.8921e-05 - accuracy: 1.0000
Epoch 8/1000
7/7 [==============================] - 0s 3ms/step - loss: 3.8770e-05 - accuracy: 1.0000
Epoch 9/1000
7/7 [==============================] - 0s 4ms/step - loss: 3.8596e-05 - accuracy: 1.0000
Epoch 10/1000
7/7 [==============================] - 0s 4ms/step - loss: 3.8439e-0

In [46]:
import pickle
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ) )

In [47]:
from keras.models import load_model
model = load_model("model.pkl")

In [48]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']

In [50]:
with open('intentss.json') as json_data:
    intents = json.load(json_data)

In [51]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
    bag=np.array(bag)
    return(bag)

In [52]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    bag = bow(sentence, words)
    results = model.predict(np.array([bag]))
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [54]:
response('Hola')

1/1 [==============================] - 0s 18ms/step
Hola! What brings you here?


In [55]:
response('Tell me more about the person who built you.')

1/1 [==============================] - 0s 17ms/step
I'm a creation of Sir Fahmi, your friendly neighborhood chatbot.


In [56]:
response('In which part of the world are you situated?')

1/1 [==============================] - 0s 25ms/step
I'm virtually located in Indonesia, serving users worldwide.


In [58]:
response("What are the types of neural network")

1/1 [==============================] - 0s 23ms/step
There are various types of neural networks, including feedforward, recurrent, convolutional, and more.


In [68]:
response("Can you explain LSTM?")

1/1 [==============================] - 0s 20ms/step
LSTM, or Long Short-Term Memory, is a type of recurrent neural network designed to overcome the vanishing gradient problem.


In [77]:
response("Explain the concept of deep learning")

1/1 [==============================] - 0s 17ms/step
Deep learning is a subset of machine learning that involves neural networks with many layers, enabling them to learn intricate patterns.


In [79]:
response("Until next time")

1/1 [==============================] - 0s 20ms/step
Adios! Have a great day ahead
